In [1]:
import pandas as pd
import requests
from tqdm import tqdm
from scrapy.http import TextResponse
from selenium import webdriver

In [2]:
url = 'https://www.seek.com.au/'
driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')
driver.get(url)

<ipython-input-2-2afe550d7d72>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')


In [5]:
start_url = driver.current_url
seek_job = {'Url':[],'Title':[],'Company':[],'Posted':[],'Text':[]}
def scrape():
    for i in tqdm(range(1, 6)):
        link = start_url+'?page='+str(i)
        res = requests.get(link)
        response = TextResponse(res.url, body=res.text, encoding='utf-8')
        links = [response.urljoin(i) for i in response.xpath('//a[@data-automation="jobTitle"]/@href').extract()]
        for j in links:
            res = requests.get(j)
            response = TextResponse(res.url, body=res.text, encoding='utf-8')
            seek_job['Url'].append(j)
            title = response.xpath('//h1[@data-automation="job-detail-title"]/text()').extract_first()
            seek_job['Title'].append(title)
            company = response.xpath('//span[@data-automation="advertiser-name"]//text()').extract_first()
            seek_job['Company'].append(company)
            posted = response.xpath('//*[contains(text(),"Posted")]/span/text()').extract_first()
            seek_job['Posted'].append(posted)
            text = ' '.join(response.xpath('//div[@data-automation="jobAdDetails"]//text()').extract())
            seek_job['Text'].append(text)
    return seek_job

In [6]:
seek_job_data = scrape()
seek_jobs_df = pd.DataFrame(seek_job_data)
seek_jobs_df.to_csv('commbank.csv',index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:30<00:00, 18.08s/it]


In [10]:
pd.Series(' '.join(seek_jobs_df['Text']).split()).value_counts().to_csv('perfekt.csv')